# Supervised Classification using TabNet

In this notebook, we will see how to use the TabNet layers from this repository to build a supervised learning classifier in TF Keras. As an example, we will analyze the [Adult Census Income dataset](https://archive.ics.uci.edu/ml/datasets/adult). The goal here is to predict the income bracket (earns <=\$50K or >50K) of an individual based on some descriptive features (e.g., age, sex, race, marital status, occupation, capital gain/loss, etc.).

In [1]:
import pandas as pd
import tensorflow as tf

## Loading and preparing data
We load the Adult Census Income dataset from the UCI data repository. We impute the missing values with the mode value. We encode the labels into binary. Finally, we convert the data into [TF Dataset format](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [2]:
train_data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
feature_names = ["age", "workclass", "fnlwgt", "education", "education-num", 
                 "marital-status", "occupation", "relationship", "race", "sex",
                 "capital-gain", "capital-loss", "hours-per-week", 
                 "native-country", "income"]

In [3]:
train_data = pd.read_csv(train_data_url, 
                         sep=",", 
                         skipinitialspace=True, 
                         na_values="?", 
                         names=feature_names)

test_data = pd.read_csv(test_data_url, 
                        sep=",", 
                        skiprows=1, 
                        skipinitialspace=True, 
                        na_values="?", 
                        names=feature_names)

In [4]:
print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

Train shape: (32561, 15)
Test shape: (16281, 15)


In [5]:
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
print("Train: missing values")
print(train_data.isnull().sum())
print("\nTest: missing values")
print(test_data.isnull().sum())

Train: missing values
age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

Test: missing values
age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64


In [7]:
def impute_missing_with_mode(dataframe, columns):
    for col in columns:
        dataframe[col].fillna(dataframe[col].mode()[0], inplace=True)

In [8]:
columns_with_missing = ['workclass', 'occupation', 'native-country']

impute_missing_with_mode(train_data, columns_with_missing)
impute_missing_with_mode(test_data, columns_with_missing)

In [9]:
print("Train: missing values")
print(train_data.isnull().sum())
print("\nTest: missing values")
print(test_data.isnull().sum())

Train: missing values
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

Test: missing values
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [10]:
train_data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [11]:
# Converting target variable into int encoded {0,1} variable
train_data["income"] = train_data["income"].astype("category").cat.codes
test_data["income"] = test_data["income"].astype("category").cat.codes

In [12]:
train_data.nunique()

age                  73
workclass             8
fnlwgt            21648
education            16
education-num        16
marital-status        7
occupation           14
relationship          6
race                  5
sex                   2
capital-gain        119
capital-loss         92
hours-per-week       94
native-country       41
income                2
dtype: int64

In [13]:
def dataframe_to_tf_dataset(dataframe:pd.DataFrame, target_name: str):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_name)
    ds = tf.data.Dataset.from_tensor_slices(
        (dict(dataframe), labels)
    )
    return ds

In [14]:
train_ds = dataframe_to_tf_dataset(train_data, "income")
test_ds = dataframe_to_tf_dataset(test_data, "income")

2021-11-20 13:40:18.766997: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# splitting data into batches
batch_size = 4_096

train_ds = train_ds.shuffle(16_384, seed=1234).batch(batch_size).prefetch(1)
test_ds = test_ds.batch(batch_size)

In [16]:
# The first TF record
for x, y in train_ds.unbatch().take(1):
    print(f"X: {x}")
    print(f"y: {y}")
    print(f"Age: {x['age']}")

X: {'age': <tf.Tensor: shape=(), dtype=int64, numpy=22>, 'workclass': <tf.Tensor: shape=(), dtype=string, numpy=b'Private'>, 'fnlwgt': <tf.Tensor: shape=(), dtype=int64, numpy=100587>, 'education': <tf.Tensor: shape=(), dtype=string, numpy=b'Some-college'>, 'education-num': <tf.Tensor: shape=(), dtype=int64, numpy=10>, 'marital-status': <tf.Tensor: shape=(), dtype=string, numpy=b'Never-married'>, 'occupation': <tf.Tensor: shape=(), dtype=string, numpy=b'Other-service'>, 'relationship': <tf.Tensor: shape=(), dtype=string, numpy=b'Own-child'>, 'race': <tf.Tensor: shape=(), dtype=string, numpy=b'Black'>, 'sex': <tf.Tensor: shape=(), dtype=string, numpy=b'Female'>, 'capital-gain': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'capital-loss': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'hours-per-week': <tf.Tensor: shape=(), dtype=int64, numpy=15>, 'native-country': <tf.Tensor: shape=(), dtype=string, numpy=b'United-States'>}
y: 0
Age: 22


## Building a Keras TabNet model